In [ ]:
import cornac
import pandas as pd

k = 20

In [ ]:
# modified from https://github.com/microsoft/recommenders/blob/98d661edc6a9965c7f42b76dc5317af3ae74d5e0/recommenders/models/cornac/cornac_utils.py

import pandas as pd
import numpy as np


# Default column names
DEFAULT_USER_COL = "user_id"
DEFAULT_ITEM_COL = "business_id"
DEFAULT_RATING_COL = "rating"
DEFAULT_LABEL_COL = "label"
DEFAULT_RELEVANCE_COL = "relevance"
DEFAULT_TIMESTAMP_COL = "timestamp"
DEFAULT_PREDICTION_COL = "prediction"
DEFAULT_SIMILARITY_COL = "sim"
COL_DICT = {
    "col_user": DEFAULT_USER_COL,
    "col_item": DEFAULT_ITEM_COL,
    "col_rating": DEFAULT_RATING_COL,
    "col_prediction": DEFAULT_PREDICTION_COL,
}

# Filtering variables
DEFAULT_K = 10
DEFAULT_THRESHOLD = 10

# Other
SEED = 42


def predict(
    model,
    data,
    usercol=DEFAULT_USER_COL,
    itemcol=DEFAULT_ITEM_COL,
    predcol=DEFAULT_PREDICTION_COL,
):
    """Computes predictions of a recommender model from Cornac on the data.
    Can be used for computing rating metrics like RMSE.

    Args:
        model (cornac.models.Recommender): A recommender model from Cornac
        data (pandas.DataFrame): The data on which to predict
        usercol (str): Name of the user column
        itemcol (str): Name of the item column

    Returns:
        pandas.DataFrame: Dataframe with usercol, itemcol, predcol
    """
    uid_map = model.train_set.uid_map
    iid_map = model.train_set.iid_map
    predictions = [
        [
            getattr(row, usercol),
            getattr(row, itemcol),
            model.rate(
                user_idx=uid_map.get(getattr(row, usercol), len(uid_map)),
                item_idx=iid_map.get(getattr(row, itemcol), len(iid_map)),
            ),
        ]
        for row in data.itertuples()
    ]
    predictions = pd.DataFrame(data=predictions, columns=[usercol, itemcol, predcol])
    return predictions


def predict_ranking(
    model,
    data,
    usercol=DEFAULT_USER_COL,
    itemcol=DEFAULT_ITEM_COL,
    predcol=DEFAULT_PREDICTION_COL,
    remove_seen=False,
):
    """Computes predictions of recommender model from Cornac on all users and items in data.
    It can be used for computing ranking metrics like NDCG.

    Args:
        model (cornac.models.Recommender): A recommender model from Cornac
        data (pandas.DataFrame): The data from which to get the users and items
        usercol (str): Name of the user column
        itemcol (str): Name of the item column
        remove_seen (bool): Flag to remove (user, item) pairs seen in the training data

    Returns:
        pandas.DataFrame: Dataframe with usercol, itemcol, predcol
    """
    users, items, preds = [], [], []
    item = list(model.train_set.iid_map.keys())
    for uid, user_idx in model.train_set.uid_map.items():
        user = [uid] * len(item)
        users.extend(user)
        items.extend(item)
        preds.extend(model.score(user_idx).tolist())

    all_predictions = pd.DataFrame(
        data={usercol: users, itemcol: items, predcol: preds}
    )

    if remove_seen:
        tempdf = pd.concat(
            [
                data[[usercol, itemcol]],
                pd.DataFrame(
                    data=np.ones(data.shape[0]), columns=["dummycol"], index=data.index
                ),
            ],
            axis=1,
        )
        merged = pd.merge(tempdf, all_predictions, on=[usercol, itemcol], how="outer")
        return merged[merged["dummycol"].isnull()].drop("dummycol", axis=1)
    else:
        return all_predictions


# map_at_k, precision_at_k, recall_at_k


def get_top_k_items(
    dataframe, col_user=DEFAULT_USER_COL, col_rating=DEFAULT_RATING_COL, k=DEFAULT_K
):
    """Get the input customer-item-rating tuple in the format of Pandas
    DataFrame, output a Pandas DataFrame in the dense format of top k items
    for each user.

    Note:
        If it is implicit rating, just append a column of constants to be
        ratings.

    Args:
        dataframe (pandas.DataFrame): DataFrame of rating data (in the format
        customerID-itemID-rating)
        col_user (str): column name for user
        col_rating (str): column name for rating
        k (int or None): number of items for each user; None means that the input has already been
        filtered out top k items and sorted by ratings and there is no need to do that again.

    Returns:
        pandas.DataFrame: DataFrame of top k items for each user, sorted by `col_user` and `rank`
    """
    # Sort dataframe by col_user and (top k) col_rating
    if k is None:
        top_k_items = dataframe
    else:
        top_k_items = (
            dataframe.sort_values([col_user, col_rating], ascending=[True, False])
            .groupby(col_user, as_index=False)
            .head(k)
            .reset_index(drop=True)
        )
    # Add ranks
    top_k_items["rank"] = top_k_items.groupby(col_user, sort=False).cumcount() + 1
    return top_k_items


def _get_rating_column(relevancy_method: str, **kwargs) -> str:
    r"""Helper utility to simplify the arguments of eval metrics
    Attemtps to address https://github.com/microsoft/recommenders/issues/1737.

    Args:
        relevancy_method (str): method for determining relevancy ['top_k', 'by_threshold', None]. None means that the
            top k items are directly provided, so there is no need to compute the relevancy operation.

    Returns:
        str: rating column name.
    """
    if relevancy_method != "top_k":
        if "col_rating" not in kwargs:
            raise ValueError("Expected an argument `col_rating` but wasn't found.")
        col_rating = kwargs.get("col_rating")
    else:
        col_rating = kwargs.get("col_rating", DEFAULT_RATING_COL)
    return col_rating


def merge_ranking_true_pred(
    rating_true,
    rating_pred,
    col_user,
    col_item,
    col_rating,
    col_prediction,
    relevancy_method,
    k=DEFAULT_K,
    threshold=DEFAULT_THRESHOLD,
):
    """Filter truth and prediction data frames on common users

    Args:
        rating_true (pandas.DataFrame): True DataFrame
        rating_pred (pandas.DataFrame): Predicted DataFrame
        col_user (str): column name for user
        col_item (str): column name for item
        col_rating (str): column name for rating
        col_prediction (str): column name for prediction
        relevancy_method (str): method for determining relevancy ['top_k', 'by_threshold', None]. None means that the
            top k items are directly provided, so there is no need to compute the relevancy operation.
        k (int): number of top k items per user (optional)
        threshold (float): threshold of top items per user (optional)

    Returns:
        pandas.DataFrame, pandas.DataFrame, int: DataFrame of recommendation hits, sorted by `col_user` and `rank`
        DataFrame of hit counts vs actual relevant items per user number of unique user ids
    """

    # Make sure the prediction and true data frames have the same set of users
    common_users = set(rating_true[col_user]).intersection(set(rating_pred[col_user]))
    rating_true_common = rating_true[rating_true[col_user].isin(common_users)]
    rating_pred_common = rating_pred[rating_pred[col_user].isin(common_users)]
    n_users = len(common_users)

    # Return hit items in prediction data frame with ranking information. This is used for calculating NDCG and MAP.
    # Use first to generate unique ranking values for each item. This is to align with the implementation in
    # Spark evaluation metrics, where index of each recommended items (the indices are unique to items) is used
    # to calculate penalized precision of the ordered items.
    if relevancy_method == "top_k":
        top_k = k
    elif relevancy_method == "by_threshold":
        top_k = threshold
    elif relevancy_method is None:
        top_k = None
    else:
        raise NotImplementedError("Invalid relevancy_method")
    df_hit = get_top_k_items(
        dataframe=rating_pred_common,
        col_user=col_user,
        col_rating=col_prediction,
        k=top_k,
    )
    df_hit = pd.merge(df_hit, rating_true_common, on=[col_user, col_item])[
        [col_user, col_item, "rank"]
    ]

    # count the number of hits vs actual relevant items per user
    df_hit_count = pd.merge(
        df_hit.groupby(col_user, as_index=False)[col_user].agg({"hit": "count"}),
        rating_true_common.groupby(col_user, as_index=False)[col_user].agg(
            {"actual": "count"}
        ),
        on=col_user,
    )

    return df_hit, df_hit_count, n_users


def precision_at_k(
    rating_true,
    rating_pred,
    col_user=DEFAULT_USER_COL,
    col_item=DEFAULT_ITEM_COL,
    col_prediction=DEFAULT_PREDICTION_COL,
    relevancy_method="top_k",
    k=DEFAULT_K,
    threshold=DEFAULT_THRESHOLD,
    **kwargs
):
    """Precision at K.

    Note:
        We use the same formula to calculate precision@k as that in Spark.
        More details can be found at
        http://spark.apache.org/docs/2.1.1/api/python/pyspark.mllib.html#pyspark.mllib.evaluation.RankingMetrics.precisionAt
        In particular, the maximum achievable precision may be < 1, if the number of items for a
        user in rating_pred is less than k.

    Args:
        rating_true (pandas.DataFrame): True DataFrame
        rating_pred (pandas.DataFrame): Predicted DataFrame
        col_user (str): column name for user
        col_item (str): column name for item
        col_rating (str): column name for rating
        col_prediction (str): column name for prediction
        relevancy_method (str): method for determining relevancy ['top_k', 'by_threshold', None]. None means that the
            top k items are directly provided, so there is no need to compute the relevancy operation.
        k (int): number of top k items per user
        threshold (float): threshold of top items per user (optional)

    Returns:
        float: precision at k (min=0, max=1)
    """
    col_rating = _get_rating_column(relevancy_method, **kwargs)
    df_hit, df_hit_count, n_users = merge_ranking_true_pred(
        rating_true=rating_true,
        rating_pred=rating_pred,
        col_user=col_user,
        col_item=col_item,
        col_rating=col_rating,
        col_prediction=col_prediction,
        relevancy_method=relevancy_method,
        k=k,
        threshold=threshold,
    )

    if df_hit.shape[0] == 0:
        return 0.0

    return (df_hit_count["hit"] / k).sum() / n_users


def recall_at_k(
    rating_true,
    rating_pred,
    col_user=DEFAULT_USER_COL,
    col_item=DEFAULT_ITEM_COL,
    col_prediction=DEFAULT_PREDICTION_COL,
    relevancy_method="top_k",
    k=DEFAULT_K,
    threshold=DEFAULT_THRESHOLD,
    **kwargs
):
    """Recall at K.

    Args:
        rating_true (pandas.DataFrame): True DataFrame
        rating_pred (pandas.DataFrame): Predicted DataFrame
        col_user (str): column name for user
        col_item (str): column name for item
        col_rating (str): column name for rating
        col_prediction (str): column name for prediction
        relevancy_method (str): method for determining relevancy ['top_k', 'by_threshold', None]. None means that the
            top k items are directly provided, so there is no need to compute the relevancy operation.
        k (int): number of top k items per user
        threshold (float): threshold of top items per user (optional)

    Returns:
        float: recall at k (min=0, max=1). The maximum value is 1 even when fewer than
        k items exist for a user in rating_true.
    """
    col_rating = _get_rating_column(relevancy_method, **kwargs)
    df_hit, df_hit_count, n_users = merge_ranking_true_pred(
        rating_true=rating_true,
        rating_pred=rating_pred,
        col_user=col_user,
        col_item=col_item,
        col_rating=col_rating,
        col_prediction=col_prediction,
        relevancy_method=relevancy_method,
        k=k,
        threshold=threshold,
    )

    if df_hit.shape[0] == 0:
        return 0.0

    return (df_hit_count["hit"] / df_hit_count["actual"]).sum() / n_users


def map_at_k(
    rating_true,
    rating_pred,
    col_user=DEFAULT_USER_COL,
    col_item=DEFAULT_ITEM_COL,
    col_prediction=DEFAULT_PREDICTION_COL,
    relevancy_method="top_k",
    k=DEFAULT_K,
    threshold=DEFAULT_THRESHOLD,
    **kwargs
):
    """Mean Average Precision at k

    The implementation of MAP is referenced from Spark MLlib evaluation metrics.
    https://spark.apache.org/docs/2.3.0/mllib-evaluation-metrics.html#ranking-systems

    A good reference can be found at:
    http://web.stanford.edu/class/cs276/handouts/EvaluationNew-handout-6-per.pdf

    Note:
        1. The evaluation function is named as 'MAP is at k' because the evaluation class takes top k items for
        the prediction items. The naming is different from Spark.

        2. The MAP is meant to calculate Avg. Precision for the relevant items, so it is normalized by the number of
        relevant items in the ground truth data, instead of k.

    Args:
        rating_true (pandas.DataFrame): True DataFrame
        rating_pred (pandas.DataFrame): Predicted DataFrame
        col_user (str): column name for user
        col_item (str): column name for item
        col_rating (str): column name for rating
        col_prediction (str): column name for prediction
        relevancy_method (str): method for determining relevancy ['top_k', 'by_threshold', None]. None means that the
            top k items are directly provided, so there is no need to compute the relevancy operation.
        k (int): number of top k items per user
        threshold (float): threshold of top items per user (optional)

    Returns:
        float: MAP at k (min=0, max=1).
    """
    col_rating = _get_rating_column(relevancy_method, **kwargs)
    df_hit, df_hit_count, n_users = merge_ranking_true_pred(
        rating_true=rating_true,
        rating_pred=rating_pred,
        col_user=col_user,
        col_item=col_item,
        col_rating=col_rating,
        col_prediction=col_prediction,
        relevancy_method=relevancy_method,
        k=k,
        threshold=threshold,
    )

    if df_hit.shape[0] == 0:
        return 0.0

    # calculate reciprocal rank of items for each user and sum them up
    df_hit_sorted = df_hit.copy()
    df_hit_sorted["rr"] = (
        df_hit_sorted.groupby(col_user).cumcount() + 1
    ) / df_hit_sorted["rank"]
    df_hit_sorted = df_hit_sorted.groupby(col_user).agg({"rr": "sum"}).reset_index()

    df_merge = pd.merge(df_hit_sorted, df_hit_count, on=col_user)
    return (df_merge["rr"] / df_merge["actual"]).sum() / n_users

In [ ]:
node_type_file_path = "node_type_ID.txt"
node_type_index = {}  # ID: type
users = []
pois = []
with open(node_type_file_path) as f:
    lines = f.readlines()
    for line in lines:
        line_content = line.strip().split("\t")
        node = line_content[0]
        node_type = line_content[1]
        node_type_index[node] = node_type

        if node_type == "P":
            pois.append(node)
        elif node_type == "U":
            users.append(node)
user_map = {node_id: i for i, node_id in enumerate(users)}
poi_map = {node_id: i for i, node_id in enumerate(pois)}
my_pred = np.load("a-pred.npy")

train = pd.read_json("review_train_fixed.json", orient="split")[
    ["user_id", "business_id"]
]
train.drop_duplicates(inplace=True)
train["r"] = 1

test = pd.read_json("review_test_fixed.json", orient="split")[
    ["user_id", "business_id"]
]
test.drop_duplicates(inplace=True)

train_set = cornac.data.Dataset.from_uir(train.itertuples(index=False, name=None))

In [ ]:
# MF

model = cornac.models.MF()
model.fit(train_set)
pred = predict_ranking(model, train)

pred["u"] = pred.apply(lambda row: user_map[row["user_id"]], axis=1)
pred["p"] = pred.apply(lambda row: poi_map[row["business_id"]], axis=1)
npy = pred.pivot(index="u", columns="p", values="prediction").to_numpy()
np.save("mf.npy", npy)

eval_map = map_at_k(test, pred, k=k)
eval_precision = precision_at_k(test, pred, k=k)
eval_recall = recall_at_k(test, pred, k=k)
eval_map, eval_precision, eval_recall

(0.0012510133449056663, 0.002951577801958651, 0.007887955211758649)

In [ ]:
# SVD

model = cornac.models.SVD()
model.fit(train_set)
pred = predict_ranking(model, train)

pred["u"] = pred.apply(lambda row: user_map[row["user_id"]], axis=1)
pred["p"] = pred.apply(lambda row: poi_map[row["business_id"]], axis=1)
npy = pred.pivot(index="u", columns="p", values="prediction").to_numpy()
np.save("svd.npy", npy)

eval_map = map_at_k(test, pred, k=k)
eval_precision = precision_at_k(test, pred, k=k)
eval_recall = recall_at_k(test, pred, k=k)
eval_map, eval_precision, eval_recall

(0.0013715142401897742, 0.003060391730141458, 0.008974847704143793)

In [ ]:
# GlobalAvg

model = cornac.models.GlobalAvg()
model.fit(train_set)
pred = predict_ranking(model, train)


eval_map = map_at_k(test, pred, k=k)
eval_precision = precision_at_k(test, pred, k=k)
eval_recall = recall_at_k(test, pred, k=k)
eval_map, eval_precision, eval_recall

(0.003307423827036806, 0.006950489662676824, 0.019476880645896505)

In [ ]:
# NMF

model = cornac.models.NMF()
model.fit(train_set)
pred = predict_ranking(model, train)


eval_map = map_at_k(test, pred, k=k)
eval_precision = precision_at_k(test, pred, k=k)
eval_recall = recall_at_k(test, pred, k=k)
eval_map, eval_precision, eval_recall

(0.007527724683373534, 0.009834058759521218, 0.03244124942006573)

In [ ]:
# Ours

pred["my_pred"] = pred.apply(
    lambda row: my_pred[user_map[row["user_id"]], poi_map[row["business_id"]]], axis=1
)

pred["u"] = pred.apply(lambda row: user_map[row["user_id"]], axis=1)
pred["p"] = pred.apply(lambda row: poi_map[row["business_id"]], axis=1)
npy = pred.pivot(index="u", columns="p", values="prediction").to_numpy()
np.save("ours.npy", npy)

eval_map = map_at_k(test, pred, k=k, col_prediction="my_pred")
eval_precision = precision_at_k(test, pred, k=k, col_prediction="my_pred")
eval_recall = recall_at_k(test, pred, k=k, col_prediction="my_pred")
eval_map, eval_precision, eval_recall

(0.008857796870751554, 0.010214907508161043, 0.03639250229293792)

In [ ]:
# NMF+Ours

pred["mix_pred"] = pred["prediction"] + 0.01 * pred["my_pred"]
eval_map = map_at_k(test, pred, k=k, col_prediction="mix_pred")

eval_precision = precision_at_k(test, pred, k=k, col_prediction="mix_pred")
eval_recall = recall_at_k(test, pred, k=k, col_prediction="mix_pred")
eval_map, eval_precision, eval_recall

(0.018461178029186346, 0.014200217627856367, 0.05016586989526903)

In [ ]:
# BPR

model = cornac.models.BPR()
model.fit(train_set)
pred = predict_ranking(model, train)

pred["u"] = pred.apply(lambda row: user_map[row["user_id"]], axis=1)
pred["p"] = pred.apply(lambda row: poi_map[row["business_id"]], axis=1)
npy = pred.pivot(index="u", columns="p", values="prediction").to_numpy()
np.save("bpr.npy", npy)

eval_map = map_at_k(test, pred, k=k)
eval_precision = precision_at_k(test, pred, k=k)
eval_recall = recall_at_k(test, pred, k=k)
eval_map, eval_precision, eval_recall

(0.023247813681894207, 0.024156692056583242, 0.08323540636308963)

In [ ]:
# BPR+Ours

pred["my_pred"] = pred.apply(
    lambda row: my_pred[user_map[row["user_id"]], poi_map[row["business_id"]]], axis=1
)
pred["mix_pred"] = pred["prediction"] + 0.5 * pred["my_pred"]

pred["u"] = pred.apply(lambda row: user_map[row["user_id"]], axis=1)
pred["p"] = pred.apply(lambda row: poi_map[row["business_id"]], axis=1)
npy = pred.pivot(index="u", columns="p", values="prediction").to_numpy()
np.save("bpr+ours.npy", npy)

eval_map = map_at_k(test, pred, k=k, col_prediction="mix_pred")
eval_precision = precision_at_k(test, pred, k=k, col_prediction="mix_pred")
eval_recall = recall_at_k(test, pred, k=k, col_prediction="mix_pred")
eval_map, eval_precision, eval_recall

(0.026438096775013277, 0.025979325353645267, 0.09256534167237288)